In [1]:
from pyspark.sql import SparkSession

Creating the spark session

In [2]:
spark = SparkSession. \
    builder. \
    enableHiveSupport(). \
    appName('Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


list databases

In [3]:
spark.sql('SHOW DATABASES').show()

+------------+
|   namespace|
+------------+
|     default|
|          hr|
|       kevin|
|kevin_retail|
|      retail|
|         sms|
|        test|
+------------+



Dropping the database if it exist

In [4]:
spark.sql('DROP DATABASE IF EXISTS kevin_retail CASCADE')

DataFrame[]

In [4]:
spark.sql('select current_database()').show()

+------------------+
|current_database()|
+------------------+
|           default|
+------------------+



creating the database

In [5]:
spark.sql('CREATE DATABASE IF NOT EXISTS kevin_retail')

DataFrame[]

In [6]:
spark.sql('use kevin_retail')

DataFrame[]

In [7]:
spark.sql('select current_database()').show()

+------------------+
|current_database()|
+------------------+
|      kevin_retail|
+------------------+



In [10]:
spark.sql('DROP TABLE  IF EXISTS orders')


DataFrame[]

## creating managed table

In [11]:
spark.sql("CREATE TABLE orders (\
    order_id INT,\
    order_date STRING,\
    order_customer_id INT,\
    order_status STRING\
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','")

DataFrame[]

loading data from hdfs to table

In [14]:
spark.sql("LOAD DATA INPATH '/data/retail_db/orders' INTO TABLE orders")

DataFrame[]

## PROJECTING DATA

In [15]:
spark.sql("SELECT * FROM orders LIMIT 10").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   34565|2014-02-23 00:00:...|             8702|       COMPLETE|
|   34566|2014-02-23 00:00:...|             3066|PENDING_PAYMENT|
|   34567|2014-02-23 00:00:...|             7314|SUSPECTED_FRAUD|
|   34568|2014-02-23 00:00:...|             1271|       COMPLETE|
|   34569|2014-02-23 00:00:...|            11083|       COMPLETE|
|   34570|2014-02-23 00:00:...|             3159|         CLOSED|
|   34571|2014-02-23 00:00:...|             4551|         CLOSED|
|   34572|2014-02-23 00:00:...|             8135|        PENDING|
|   34573|2014-02-23 00:00:...|             7497|PENDING_PAYMENT|
|   34574|2014-02-23 00:00:...|             1868|        ON_HOLD|
+--------+--------------------+-----------------+---------------+



creating table

In [16]:
spark.sql("CREATE TABLE order_items (\
    order_item_id INT,\
    order_item_order_id INT,\
    order_item_product_id INT,\
    order_item_quantity INT,\
    order_item_subtotal FLOAT,\
    order_item_product_price FLOAT\
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','")

DataFrame[]

loading data from hdfs to table

In [21]:
spark.sql("LOAD DATA INPATH '/data/retail_db/order_items' INTO TABLE order_items")

DataFrame[]

In [8]:
spark.sql("SELECT * FROM order_items LIMIT 10").show()

+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_quantity|order_item_subtotal|order_item_product_price|
+-------------+-------------------+---------------------+-------------------+-------------------+------------------------+
|            1|                  1|                  957|                  1|             299.98|                  299.98|
|            2|                  2|                 1073|                  1|             199.99|                  199.99|
|            3|                  2|                  502|                  5|              250.0|                    50.0|
|            4|                  2|                  403|                  1|             129.99|                  129.99|
|            5|                  4|                  897|                  2|              49.98|                   24.99|
|            6| 

In [9]:
spark.sql('show tables').show()

+------------+-----------+-----------+
|   namespace|  tableName|isTemporary|
+------------+-----------+-----------+
|kevin_retail|order_items|      false|
|kevin_retail|     orders|      false|
+------------+-----------+-----------+



In [24]:
spark.sql("SELECT count(1) FROM orders").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [48]:
spark.sql("SELECT count(1) FROM order_items").show()

+--------+
|count(1)|
+--------+
|  172198|
+--------+



In [10]:
spark.sql("SELECT order_customer_id, date_format(order_date, 'yyyy-MM') as order_time, order_status FROM orders LIMIT 10").show()

+-----------------+----------+---------------+
|order_customer_id|order_time|   order_status|
+-----------------+----------+---------------+
|             8702|   2014-02|       COMPLETE|
|             3066|   2014-02|PENDING_PAYMENT|
|             7314|   2014-02|SUSPECTED_FRAUD|
|             1271|   2014-02|       COMPLETE|
|            11083|   2014-02|       COMPLETE|
|             3159|   2014-02|         CLOSED|
|             4551|   2014-02|         CLOSED|
|             8135|   2014-02|        PENDING|
|             7497|   2014-02|PENDING_PAYMENT|
|             1868|   2014-02|        ON_HOLD|
+-----------------+----------+---------------+



In [29]:
spark.sql("SELECT count(1) FROM orders WHERE order_status = 'COMPLETE'").show()

+--------+
|count(1)|
+--------+
|   22899|
+--------+



In [14]:
spark.sql("SELECT count(1) FROM orders WHERE order_status IN ('COMPLETE', 'CLOSED')").show()

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [11]:
spark.sql("SELECT DISTINCT order_status FROM orders").show()

+---------------+
|   order_status|
+---------------+
|PENDING_PAYMENT|
|       COMPLETE|
|        ON_HOLD|
| PAYMENT_REVIEW|
|     PROCESSING|
|         CLOSED|
|SUSPECTED_FRAUD|
|        PENDING|
|       CANCELED|
+---------------+



In [12]:
spark.sql("SELECT DISTINCT * FROM orders").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|     189|2013-07-26 00:00:...|            10177|        PENDING|
|     379|2013-07-27 00:00:...|             8735|PENDING_PAYMENT|
|     440|2013-07-27 00:00:...|             7290|       COMPLETE|
|     471|2013-07-27 00:00:...|            10861|        PENDING|
|     599|2013-07-28 00:00:...|             8005|       COMPLETE|
|     665|2013-07-28 00:00:...|             5648|        ON_HOLD|
|     736|2013-07-29 00:00:...|             8536|       COMPLETE|
|     974|2013-07-30 00:00:...|             6413|         CLOSED|
|    1052|2013-07-30 00:00:...|              464|     PROCESSING|
|    1175|2013-07-31 00:00:...|             1330|       COMPLETE|
|    1241|2013-07-31 00:00:...|             7430|       COMPLETE|
|    1252|2013-07-31 00:00:...|            10504|        PENDING|
|    1296|

## Filtering Data

In [13]:
spark.sql("SELECT * FROM orders WHERE order_status = 'COMPLETE'").show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      22|2013-07-25 00:00:...|              333|    COMPLETE|
|      26|2013-07-25 00:00:...|             7562|    COMPLETE|
|      28|2013-07-25 00:00:...|              656|    COMPLETE|
|      32|2013-07-25 00:00:...|             3960|    COMPLETE|
|      35|2013-07-25 00:00:...|             4840|    COMPLETE|
|      45|2013-07-25 00:00:...|             2636|    COMPLETE|
|      56|2013-07-25 00:00:...|            10519|    CO

In [15]:
spark.sql("SELECT * \
    FROM orders\
    WHERE order_status = 'COMPLETE' OR order_status = 'CLOSED' LIMIT 10")\
    .show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|       1|2013-07-25 00:00:...|            11599|      CLOSED|
|       3|2013-07-25 00:00:...|            12111|    COMPLETE|
|       4|2013-07-25 00:00:...|             8827|      CLOSED|
|       5|2013-07-25 00:00:...|            11318|    COMPLETE|
|       6|2013-07-25 00:00:...|             7130|    COMPLETE|
|       7|2013-07-25 00:00:...|             4530|    COMPLETE|
|      12|2013-07-25 00:00:...|             1837|      CLOSED|
|      15|2013-07-25 00:00:...|             2568|    COMPLETE|
|      17|2013-07-25 00:00:...|             2667|    COMPLETE|
|      18|2013-07-25 00:00:...|             1205|      CLOSED|
+--------+--------------------+-----------------+------------+



In [ ]:
spark.sql("SELECT * FROM orders WHERE order_status = 'COMPLETE'").show()

In [41]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
""").show()

+--------+--------------------+---------------+-------------------+
|order_id|          order_date|   order_status|order_item_subtotal|
+--------+--------------------+---------------+-------------------+
|       1|2013-07-25 00:00:...|         CLOSED|             299.98|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             199.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|              250.0|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             129.99|
|       4|2013-07-25 00:00:...|         CLOSED|              49.98|
|       4|2013-07-25 00:00:...|         CLOSED|             299.95|
|       4|2013-07-25 00:00:...|         CLOSED|              150.0|
|       4|2013-07-25 00:00:...|         CLOSED|             199.92|
|       5|2013-07-25 00:00:...|       COMPLETE|             299.98|
|       5|2013-07-25 00:00:...|       COMPLETE|             299.95|
|       5|2013-07-25 00:00:...|       COMPLETE|              99.96|
|       5|2013-07-25 00:00:...|       COMPLETE| 

In [42]:
spark.sql("""
SELECT count(1)
FROM orders
""").show()

+--------+
|count(1)|
+--------+
|   68883|
+--------+



In [16]:
spark.sql("SELECT count(1) FROM orders WHERE order_status = 'COMPLETE'").show()

+--------+
|count(1)|
+--------+
|   22899|
+--------+



In [17]:
spark.sql("""
SELECT count(1) FROM orders 
WHERE order_status IN ('COMPLETE', 'CLOSED')
""").show()

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [18]:
spark.sql("""
SELECT count(1) FROM orders 
WHERE order_status IN ('COMPLETE', 'CLOSED')
    AND order_date LIKE '2014-01%'
""").show()

+--------+
|count(1)|
+--------+
|    2544|
+--------+



In [19]:
spark.sql("""
SELECT * FROM orders 
WHERE order_status IN ('COMPLETE', 'CLOSED')
    AND date_format(order_date, 'yyyy-MM') = '2014-01'
""").show()

+--------+--------------------+-----------------+------------+
|order_id|          order_date|order_customer_id|order_status|
+--------+--------------------+-----------------+------------+
|   25882|2014-01-01 00:00:...|             4598|    COMPLETE|
|   25888|2014-01-01 00:00:...|             6735|    COMPLETE|
|   25889|2014-01-01 00:00:...|            10045|    COMPLETE|
|   25891|2014-01-01 00:00:...|             3037|      CLOSED|
|   25895|2014-01-01 00:00:...|             1044|    COMPLETE|
|   25897|2014-01-01 00:00:...|             6405|    COMPLETE|
|   25898|2014-01-01 00:00:...|             3950|    COMPLETE|
|   25899|2014-01-01 00:00:...|             8068|      CLOSED|
|   25900|2014-01-01 00:00:...|             2382|      CLOSED|
|   25901|2014-01-01 00:00:...|             3099|    COMPLETE|
|   25902|2014-01-01 00:00:...|             6083|    COMPLETE|
|   25907|2014-01-01 00:00:...|             3603|    COMPLETE|
|   25913|2014-01-01 00:00:...|             9382|    CO

## Joining Tables - Inner

In [20]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
""").show()

+--------+--------------------+---------------+-------------------+
|order_id|          order_date|   order_status|order_item_subtotal|
+--------+--------------------+---------------+-------------------+
|       1|2013-07-25 00:00:...|         CLOSED|             299.98|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             199.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|              250.0|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|             129.99|
|       4|2013-07-25 00:00:...|         CLOSED|              49.98|
|       4|2013-07-25 00:00:...|         CLOSED|             299.95|
|       4|2013-07-25 00:00:...|         CLOSED|              150.0|
|       4|2013-07-25 00:00:...|         CLOSED|             199.92|
|       5|2013-07-25 00:00:...|       COMPLETE|             299.98|
|       5|2013-07-25 00:00:...|       COMPLETE|             299.95|
|       5|2013-07-25 00:00:...|       COMPLETE|              99.96|
|       5|2013-07-25 00:00:...|       COMPLETE| 

In [44]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_subtotal
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
    AND date_format(order_date, 'yyyy-MM') = '2014-01'
""").show()

+--------+--------------------+------------+-------------------+
|order_id|          order_date|order_status|order_item_subtotal|
+--------+--------------------+------------+-------------------+
|   25882|2014-01-01 00:00:...|    COMPLETE|             299.97|
|   25882|2014-01-01 00:00:...|    COMPLETE|              100.0|
|   25882|2014-01-01 00:00:...|    COMPLETE|              79.98|
|   25882|2014-01-01 00:00:...|    COMPLETE|             399.98|
|   25888|2014-01-01 00:00:...|    COMPLETE|             299.98|
|   25889|2014-01-01 00:00:...|    COMPLETE|              99.96|
|   25889|2014-01-01 00:00:...|    COMPLETE|              19.99|
|   25891|2014-01-01 00:00:...|      CLOSED|              150.0|
|   25891|2014-01-01 00:00:...|      CLOSED|               50.0|
|   25891|2014-01-01 00:00:...|      CLOSED|             119.97|
|   25895|2014-01-01 00:00:...|    COMPLETE|             399.98|
|   25895|2014-01-01 00:00:...|    COMPLETE|             199.99|
|   25897|2014-01-01 00:0

## Joining Tables - Outer

In [21]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_order_id,
    oi.order_item_subtotal
FROM orders o LEFT OUTER JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
""").show()

+--------+--------------------+---------------+-------------------+-------------------+
|order_id|          order_date|   order_status|order_item_order_id|order_item_subtotal|
+--------+--------------------+---------------+-------------------+-------------------+
|       1|2013-07-25 00:00:...|         CLOSED|                  1|             299.98|
|       1|2013-07-25 00:00:...|         CLOSED|                  1|             299.98|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|             129.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|              250.0|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|             199.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|             129.99|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|              250.0|
|       2|2013-07-25 00:00:...|PENDING_PAYMENT|                  2|             199.99|
|       3|2013-07-25 00:00:...| 

In [22]:
spark.sql("""
SELECT count(1)
FROM orders o LEFT OUTER JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
""").show()

+--------+
|count(1)|
+--------+
|  355848|
+--------+



In [23]:
spark.sql("""
SELECT o.order_id,
    o.order_date,
    o.order_status,
    oi.order_item_order_id,
    oi.order_item_subtotal
FROM orders o LEFT OUTER JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE oi.order_item_order_id IS NULL
""").show()

+--------+--------------------+---------------+-------------------+-------------------+
|order_id|          order_date|   order_status|order_item_order_id|order_item_subtotal|
+--------+--------------------+---------------+-------------------+-------------------+
|       3|2013-07-25 00:00:...|       COMPLETE|               null|               null|
|       6|2013-07-25 00:00:...|       COMPLETE|               null|               null|
|      22|2013-07-25 00:00:...|       COMPLETE|               null|               null|
|      26|2013-07-25 00:00:...|       COMPLETE|               null|               null|
|      32|2013-07-25 00:00:...|       COMPLETE|               null|               null|
|      40|2013-07-25 00:00:...|PENDING_PAYMENT|               null|               null|
|      47|2013-07-25 00:00:...|PENDING_PAYMENT|               null|               null|
|      53|2013-07-25 00:00:...|     PROCESSING|               null|               null|
|      54|2013-07-25 00:00:...|P

## Aggregating Data

In [24]:
spark.sql("SELECT count(order_id) FROM orders").show()

+---------------+
|count(order_id)|
+---------------+
|          68883|
+---------------+



In [25]:
spark.sql("SELECT count(DISTINCT order_date) FROM orders").show()

+--------------------------+
|count(DISTINCT order_date)|
+--------------------------+
|                       364|
+--------------------------+



In [26]:
spark.sql("""
SELECT round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items 
WHERE order_item_order_id = 2
""").show()

+-------------+
|order_revenue|
+-------------+
|      1159.96|
+-------------+



In [27]:
spark.sql("""
SELECT count(1) 
FROM orders
WHERE order_status IN ('COMPLETE', 'CLOSED')
""").show()

+--------+
|count(1)|
+--------+
|   30455|
+--------+



In [36]:
spark.sql("""
SELECT date_format(order_date,'yyyy-MM') as date,
    count(1) As count
FROM orders
GROUP BY order_date
""").show(truncate=False)

+-------+-----+
|date   |count|
+-------+-----+
|2013-08|73   |
|2013-10|162  |
|2013-11|135  |
|2013-09|121  |
|2013-09|139  |
|2013-12|266  |
|2013-09|276  |
|2013-12|170  |
|2014-01|163  |
|2013-10|139  |
|2013-11|246  |
|2014-01|163  |
|2014-01|158  |
|2014-02|174  |
|2013-09|215  |
|2013-09|109  |
|2013-10|154  |
|2014-01|207  |
|2014-02|151  |
|2014-02|221  |
+-------+-----+
only showing top 20 rows



In [38]:
spark.sql("""
SELECT order_item_order_id,
    round(sum(order_item_subtotal), 2) AS order_revenue
FROM order_items
GROUP BY order_item_order_id
""").show()

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|                148|       959.98|
|                463|      1659.84|
|                471|       339.96|
|                496|        883.9|
|               1088|       499.94|
|               1580|        599.9|
|               1591|       879.72|
|               1645|      3019.58|
|               2366|       599.94|
|               2659|      1449.82|
|               2866|      1139.92|
|               3175|       419.94|
|               3749|       287.94|
|               3794|        599.9|
|               3918|      1659.86|
|               3997|       1159.9|
|               4101|       259.98|
|               4519|       159.96|
|               4818|       799.96|
|               4900|       359.94|
+-------------------+-------------+
only showing top 20 rows



## Sorting Data

In [41]:
spark.sql("""
SELECT * FROM orders
ORDER BY order_customer_id
""").show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|   22945|2013-12-13 00:00:...|                1|       COMPLETE|
|   57963|2013-08-02 00:00:...|                2|        ON_HOLD|
|   67863|2013-11-30 00:00:...|                2|       COMPLETE|
|   33865|2014-02-18 00:00:...|                2|       COMPLETE|
|   15192|2013-10-29 00:00:...|                2|PENDING_PAYMENT|
|   22646|2013-12-11 00:00:...|                3|       COMPLETE|
|   56178|2014-07-15 00:00:...|                3|        PENDING|
|   57617|2014-07-24 00:00:...|                3|       COMPLETE|
|   61453|2013-12-14 00:00:...|                3|       COMPLETE|
|   23662|2013-12-19 00:00:...|                3|       COMPLETE|
|   35158|2014-02-26 00:00:...|                3|       COMPLETE|
|   46399|2014-05-09 00:00:...|                3|     PROCESSING|
|   37878|

In [45]:
spark.sql("""
SELECT o.order_date,
    oi.order_item_product_id,
    round(sum(oi.order_item_subtotal), 2) AS revenue
FROM orders o JOIN order_items oi
    ON o.order_id = oi.order_item_order_id
WHERE o.order_status IN ('COMPLETE', 'CLOSED')
GROUP BY o.order_date,
    oi.order_item_product_id
ORDER BY o.order_date,
    revenue DESC
""").show(truncate=False)

+---------------------+---------------------+--------+
|order_date           |order_item_product_id|revenue |
+---------------------+---------------------+--------+
|2013-07-25 00:00:00.0|1004                 |11199.44|
|2013-07-25 00:00:00.0|191                  |10198.98|
|2013-07-25 00:00:00.0|957                  |8999.4  |
|2013-07-25 00:00:00.0|365                  |6718.88 |
|2013-07-25 00:00:00.0|1073                 |5999.7  |
|2013-07-25 00:00:00.0|1014                 |5597.76 |
|2013-07-25 00:00:00.0|403                  |3899.7  |
|2013-07-25 00:00:00.0|502                  |3300.0  |
|2013-07-25 00:00:00.0|627                  |2159.46 |
|2013-07-25 00:00:00.0|226                  |1199.98 |
|2013-07-25 00:00:00.0|24                   |639.92  |
|2013-07-25 00:00:00.0|821                  |415.92  |
|2013-07-25 00:00:00.0|625                  |399.98  |
|2013-07-25 00:00:00.0|705                  |239.98  |
|2013-07-25 00:00:00.0|572                  |239.94  |
|2013-07-2